In [ ]:
import dash
from dash.dependencies import Output, Input, State
import dash_core_components as dcc
import dash_html_components as html
import plotly
import random
import plotly.graph_objs as go
from collections import deque
import sqlite3
import pandas as pd
import jupyterlab_dash

app = dash.Dash(__name__)
app.layout = html.Div(
    [html.Div([   html.H1('League Of Legends Live Twitter Sentiment'),
        dcc.Input(id='sentiment_term', value='sett', type='text'),
        dcc.Graph(id='live-graph', animate=True),
        dcc.Interval(
            id='graph-update',
            interval=1*10
        ),
     ]),
    html.Div([   html.H2('Aphelios Live Twitter Sentiment'),
        dcc.Input(id='sentiment_term2', value='Aphelios', type='text'),
        dcc.Graph(id='live-graph2', animate=True),
        dcc.Interval(
            id='graph-update2',
            interval=1*10
        ),
     ]),
    ])
    
#      html.H2('League of Legends Live Twitter Sentiment on Average'),
#         dcc.Input(id='sentiment_term', value='leagueoflegends', type='text'),
#         dcc.Graph(id='live-graph', animate=True),
#         dcc.Interval(
#             id='graph-update',
#             interval=1*10
#         ),
#       html.H2('Aphelios Live Twitter Sentiment on Average'),
#         dcc.Input(id='sentiment_term2', value='Aphelios', type='text'),
#         dcc.Graph(id='live-graph', animate=True),
#         dcc.Interval(
#             id='graph-update',
#             interval=1*10
#         ),
#     ]

@app.callback(Output('live-graph', 'figure'),
#               [Input('graph-update', 'interval')],
              [Input(component_id='sentiment_term', component_property='value')])
#              [State('graph-update', 'interval')])
def update_graph_scatter(sentiment_term):
    try:
        conn = sqlite3.connect('twitter_lol_TB.db')
        c = conn.cursor()
        df = pd.read_sql("SELECT * FROM sentiment WHERE tweet LIKE ? ORDER BY unix DESC LIMIT 10000", conn ,params=('%' + sentiment_term + '%',))
        df.sort_values('unix', inplace=True)
        df['sentiment_smoothed'] = df['sentiment'].rolling(int(len(df)/2)).mean()

        df['date'] = pd.to_datetime(df['unix'],unit='ms')
        df.set_index('date', inplace=True)

        df = df.resample('10ms').mean()
        df.dropna(inplace=True)
        X = df.index
        Y = df.sentiment
        Y2 = df.sentiment_smoothed


        data = plotly.graph_objs.Scatter(
                x=X,
                y=Y,
                name='Scatter',
                mode= 'lines+markers'
                )

        return {'data': [data],'layout' : go.Layout(xaxis=dict(range=[min(X),max(X)]),
                                                    yaxis=dict(range=[min(Y),max(Y)]),
                                                    title='Term: {}'.format(sentiment_term))}
        data2 = plotly.graph_objs.Scatter(
                x=X,
                y=Y2,
                name='Scatter',
                mode= 'lines+markers'
                )

        return {'data': [data],'layout' : go.Layout(xaxis=dict(range=[min(X),max(X)]),
                                                    yaxis=dict(range=[min(Y2),max(Y2)]),
                                                    title='Term: {}'.format(sentiment_term2))}


    except Exception as e:
        with open('errors.txt','a') as f:
            f.write(str(e))
            f.write('\n')
# viewer = jupyterlab_dash.AppViewer()            
            
# viewer.show(app)

if __name__ == '__main__':
    app.run_server(host="0.0.0.0", port=9990, debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:9990/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Dec/2019 10:41:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2019 10:41:41] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2019 10:41:41] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2019 10:41:45] "POST /_dash-update-component HTTP/1.1" 200 -


Go to this link to look at the visualization -- http://0.0.0.0:9990/ or http://localhost:9990/


Please note that the gaps are the times during the day/night that I did not collect the data. Perhaps, at some time in the future, I would like to run it on a server that continually collects the data and prunes the database to restrict the data for a certain 

There is more work to be done here. Specifically, 
a. I might work on a better UX front-end.
b. I can deploy it as a web app where users can chose which term they want to track and inspect.
c. I can show the tweet of interest on hover (raw tweet and the translated tweet where applicable).
d. I can create a visual map of where the users are tweeting from. Each tweet shows up as a color coded dot (that reflects brightness as an indicator for recency of the tweet.)
e. I can include more indicators of the tweet's perception, engagement, and reach (specifically for the tweets with the most impact, most reach, and highest polarity.)